# Chapter 4 — Lifecycle Events (Hooks) 
## 1. Quick Intro

**Hooks = lifecycle callbacks.**  
They fire at key points of an agent’s run (start, end, etc.), so you can **log, prefetch, audit, cache**, or trigger side effects. Attach them with the `hooks` argument on an `Agent`, or use `RunHooks` for whole-run visibility.

## When to use hooks
- **Observability:** timing, tool usage, handoffs, output size
- **Prefetch & caching:** warm up resources on start, persist on end
- **Compliance/Audit:** redact & store minimal traces
- **A/B & metrics:** compare prompts/models across runs

## 2. Overview — Lifecycle Hooks

There are **two families** of lifecycle hooks:

1) **Agent Hooks** — attach to a specific `Agent`.  
   Only fire when **that agent** hits a lifecycle event.

2) **Run Hooks** — attach to the **Runner** (the whole workflow).  
   Fire whenever **any agent** hits a lifecycle event during the run.

### Agent hooks (per-agent)
- **`on_start(context, agent)`** — this agent begins execution  
- **`on_end(context, agent, output)`** — this agent finishes (final output available)  
- **`on_handoff(context, agent, target_agent, input)`** — this agent hands off to another  
- **`on_tool_start(context, agent, tool_name, args)`** — a tool call begins  
- **`on_tool_end(context, agent, tool_name, result)`** — a tool call ends

### Run hooks (whole workflow)
- **`on_agent_start(context, agent)`** — any agent starts  
- **`on_agent_end(context, agent, output)`** — any agent ends  
- **`on_handoff(context, from_agent, to_agent, input)`** — any handoff occurs  
- **`on_tool_start(context, agent, tool_name, args)`** — any tool call begins  
- **`on_tool_end(context, agent, tool_name, result)`** — any tool call ends

> Use **Agent hooks** for per-agent behaviors (custom logging, prefetch, caching).  
> Use **Run hooks** for end-to-end analytics, auditing, or tracing across **all** agents.

### Wiring (tiny sketch)
```python
from agents import Agent, Runner, AgentHooks, RunHooks

class MyAgentHooks(AgentHooks):
    async def on_start(self, ctx, agent): print(f"[A] {agent.name} start")
    async def on_end(self, ctx, agent, output): print(f"[A] {agent.name} end -> {output.final_output}")

class MyRunHooks(RunHooks):
    async def on_agent_start(self, ctx, agent): print(f"[RUN] {agent.name} start")
    async def on_agent_end(self, ctx, agent, output): print(f"[RUN] {agent.name} end")

agent = Agent(name="Triage", instructions="...", model=llm, hooks=MyAgentHooks())
result = await Runner.run(agent, "Hello", hooks=MyRunHooks())  # run-level hooks here


## 3. Start & End hooks (the tiniest useful example)

We’ll attach two **Agent hooks** to a single agent:
- `on_start` → fire when this agent begins
- `on_end` → fire when this agent finishes (we’ll print a short summary)

In [12]:
from agents import Agent, Runner, AgentHooks, RunContextWrapper, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os
import asyncio
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('API_KEY')
base_url = "https://api.openai.com/v1"  
chat_model = "gpt-4.1-nano-2025-04-14"  

set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

# --- minimal hooks: count + print ---
class MyAgentHooks(AgentHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} started")

    async def on_end(self, context: RunContextWrapper, agent: Agent, output) -> None:
        self.event_counter += 1
        print(
            f"{self.event_counter}: Agent {agent. name} ended with output {output}"
        )

agent = Agent(name="Assistant", model=llm, hooks=MyAgentHooks(), instructions="You are a helpful assistant")


result = await Runner.run(agent, "what is your name?")
print(result.final_output)


1: Agent Assistant started
2: Agent Assistant ended with output I am called ChatGPT. How can I assist you today?
I am called ChatGPT. How can I assist you today?


## 4. Run-level hooks (observe the whole workflow)

**Run hooks** fire for **any agent** in the run. The API mirrors agent hooks, but names are prefixed with `on_agent_*`.  
Mount them by passing `hooks=MyRunHooks()` to `Runner.run(...)`.

- Agent hooks: `on_start`, `on_end`
- Run hooks: **`on_agent_start`**, **`on_agent_end`** (plus `on_handoff`, `on_tool_start`, `on_tool_end`, etc.)

### Minimal example

In [13]:
from agents import RunHooks


set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

class MyRunHooks(RunHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_agent_start(self, context: RunContextWrapper, agent: Agent) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} started")

    async def on_agent_end(self, context: RunContextWrapper, agent: Agent, output) -> None:
        self.event_counter += 1
        print(
            f"{self.event_counter}: Agent {agent.name} ended with output {output}"
        )

agent = Agent(name="Assistant", model=llm, instructions="You are a helpful assistant")

result = await Runner.run(agent, hooks=MyRunHooks(), input="what is your name?")
print(result.final_output)


1: Agent Assistant started
2: Agent Assistant ended with output Hello! I am an AI language model created by OpenAI. You can call me ChatGPT. How can I assist you today?
Hello! I am an AI language model created by OpenAI. You can call me ChatGPT. How can I assist you today?


#### Notes

- Use Run hooks when you want end-to-end observability across multiple agents/handoffs.

- Keep hooks lightweight (avoid blocking I/O); log summaries (output.final_output) instead of large objects.

## 5. Tool usage hooks (start & end)

Both **AgentHooks** and **RunHooks** expose the same tool events with the same signatures:

- `on_tool_start(context, agent, tool)` — fires when a tool call begins  
- `on_tool_end(context, agent, tool, output)` — fires when a tool call finishes

Below we log which tool was used and what it returned.

### Minimal example
```python

on_tool_start(
    context: RunContextWrapper[TContext],
    agent: Agent[TContext],
    tool: Tool,
) -> None

on_tool_end(
    context: RunContextWrapper[TContext],
    agent: Agent[TContext],
    tool: Tool,
    result: str,
) -> None

```

In [ ]:
from agents import function_tool,Tool


set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)

# --- a simple tool ---
@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny"

# --- hooks that log tool lifecycle ---
class MyAgentHooks(AgentHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool: Tool) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} started to use Tool {tool.name}")

    async def on_tool_end(self, context: RunContextWrapper, agent: Agent, tool: Tool, output) -> None:
        self.event_counter += 1
        print(
            f"{self.event_counter}: Agent {agent.name} ended to use Tool {tool.name} with output {output}"
        )

agent = Agent(
    name="Weather Assistant",
    instructions = (
    "Answer in the tone of Sir Humphrey Appleby. "
    "If the user asks about going out / outdoors / suitability of plans in a CITY, "
    "you MUST call the `get_weather` tool with that city first, then base your answer on the result."
    ),
    model=llm,
    tools=[get_weather],
    hooks=MyAgentHooks(),
)

result = await Runner.run(agent, "Check the weather in Paris and tell me if it's good to go out.")
print(result.final_output)

1: Agent Weather Assistant started to use Tool get_weather
2: Agent Weather Assistant ended to use Tool get_weather with output The weather in Paris is sunny
Ah, splendid news! With such fine weather gracing Paris, it appears most agreeable for outdoor pursuits. A good occasion to step outside and enjoy the day, wouldn't you say?


Because `RunHooks` uses the **same event names & signatures** as `AgentHooks`, you only change:
1) the **base class** you inherit (`RunHooks`), and  
2) where you **mount** it (pass `hooks=MyRunHooks()` to `Runner.run(...)`).

In [16]:
from agents import Agent, RunHooks


@function_tool
def get_weather(city: str) -> str:
    return f"The weather in {city} is sunny"

class MyRunHooks(RunHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_tool_start(self, context: RunContextWrapper, agent: Agent, tool: Tool) -> None:
        self.event_counter += 1
        print(f"{self.event_counter}: Agent {agent.name} started to use Tool {tool.name}")

    async def on_tool_end(self, context: RunContextWrapper, agent: Agent, tool: Tool, output) -> None:
        self.event_counter += 1
        print(
            f"{self.event_counter}: Agent {agent.name} ended to use Tool {tool.name} with output {output}"
        )

agent = Agent(
    name="Weather Assistant",
    instructions = (
    "Answer in the tone of Sir Humphrey Appleby. "
    "If the user asks about going out / outdoors / suitability of plans in a CITY, "
    "you MUST call the `get_weather` tool with that city first, then base your answer on the result."
    ),
    model=llm,
    tools=[get_weather],
)


result = await Runner.run(agent, hooks=MyRunHooks(), input="Check the weather in Paris and tell me if it's good to go out.")
print(result.final_output)


1: Agent Weather Assistant started to use Tool get_weather
2: Agent Weather Assistant ended to use Tool get_weather with output The weather in Paris is sunny
Ah, splendid! With the sun shining brightly in Paris, it would indeed be an opportune moment for outdoor activities. Do remember, however, to consider any personal comfort or specific plans you might have in mind. But generally speaking, a sunny day in Paris is most conducive to stepping outdoors and enjoying the splendid cityscape.


## 6. Handoffs (delegation) — hook it!

Handoffs have a single lifecycle event, **`on_handoff`**, but its **parameters differ** from start/end/tool hooks: you get both the **target agent** and the **source agent** so you can log *who delegated to whom*.

In [ ]:
class MyAgentHooks(AgentHooks):
    def __init__(self):
        self.event_counter = 0
        
    # Fired when another agent hands off TO this agent
    async def on_handoff(self, context: RunContextWrapper, agent: Agent, source: Agent) -> None:
        self.event_counter += 1
        print(
            f"{self.event_counter}: Agent {source.name} handed off to {agent.name}"
        )

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llm,
    hooks=MyAgentHooks(),
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
    hooks=MyAgentHooks(),
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    model=llm,
    hooks=MyAgentHooks(),
)

result = await Runner.run(triage_agent, "What is the Central Limit Theorem")
print(result.final_output)

result = await Runner.run(triage_agent, "Tell me about Lenin")
print(result.final_output)


1: Agent Triage Agent handed off to Math Tutor
The Central Limit Theorem (CLT) is a fundamental concept in statistics. It states that if you take sufficiently large random samples from a population with any distribution (the original distribution can be skewed, uniform, or any shape), the distribution of the sample means will tend to be approximately normally distributed. 

### Breaking down the key points:

1. **Sample Means:** It concerns the distribution of the averages (means) of multiple samples.
2. **Large Sample Size:** The theorem requires that the sample size be sufficiently large, commonly n ≥ 30 is considered enough, though the actual number may vary depending on the original distribution.
3. **Population distribution shape:** The original population can have any shape, but the distribution of the sample means will approximate a normal distribution as the sample size grows.

### Why is this important?

Because many statistical procedures assume normality, the CLT allows us t

## 6-bis. Handoffs with `RunHooks` (note the parameter change)

When you switch from **AgentHooks** to **RunHooks**, the **handoff** event’s signature changes:

- **AgentHooks**: `on_handoff(context, agent, source)`  
  → Fires **on the target agent**; you get the **target** (`agent`) and the **source** (`source`).

- **RunHooks**: `on_handoff(context, from_agent, to_agent)`  
  → Fires for **any handoff in the run**; you get both ends explicitly: **from** (source) and **to** (target).


In [18]:
from agents import RunHooks


class MyRunHooks(RunHooks):
    def __init__(self):
        self.event_counter = 0

    async def on_handoff(
        self, context: RunContextWrapper, from_agent: Agent, to_agent: Agent
    ) -> None:
        self.event_counter += 1
        print(
            f"### {self.event_counter}: Handoff from {from_agent.name} to {to_agent.name}."
        )

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
    model=llm,
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide help with math problems. Explain your reasoning at each step and include examples",
    model=llm,
)

triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question",
    handoffs=[history_tutor_agent, math_tutor_agent],
    model=llm,
)


result = await Runner.run(triage_agent, hooks=MyRunHooks(), input="What is the Central Limit Theorem")
print(result.final_output)

result = await Runner.run(triage_agent, hooks=MyRunHooks(), input="Tell me about Lenin?")
print(result.final_output)


### 1: Handoff from Triage Agent to Math Tutor.
The Central Limit Theorem (CLT) is a fundamental concept in statistics. It states that the sampling distribution of the sample mean will tend to be approximately normal (bell-shaped), regardless of the shape of the population distribution, as long as the sample size is sufficiently large.

Let's break this down step-by-step:

### Key points of the CLT:
1. **Sample Size**: The larger the sample size (usually n ≥ 30 is considered sufficient), the more the distribution of the sample mean will resemble a normal distribution.
2. **Population Distribution**: The original population from which samples are drawn can have any shape—skewed, bimodal, uniform, etc.
3. **Sample Means**: When you take many samples of size n from the population and compute their means, these means will tend to follow a normal distribution.

### Why is this important?
- It allows us to make inferences about the population mean even if the population distribution is not n